In [ ]:
import numpy as np
import scanpy as sc
import seaborn as sns
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import sys
import pickle as pkl
path_helper = ["C:\\","Users","vfriedrich","projects","monkey_IZI","git_documentation","scRNAseq_cross_species_primate_human","analysis","helper"]
sys.path.append(os.path.join(*path_helper))
import helperVDF as h
#import decoupler
print(sys.executable)
import gseapy
from gseapy.plot import barplot, dotplot
warnings.filterwarnings("ignore")
import re

In [ ]:
pre = "MH115"
drive = 'F'
base_model_path,base_table_path,base_plots_path,base_anndata_objects = h.return_local_paths(drive = drive,
                                                                                            pre = pre,
                                                                                            add_path = True)
def remove_go(term):
    # Use regular expression to find and replace GO terms, ensuring the string is treated as a raw string
    new_term = re.sub(r'\s*\(GO:\d+\)', '', term)  # Corrected regex finds the GO term pattern and removes it
    return new_term.strip() 

col_human =  "#e35e28"
col_cyno ="#31c7ba"
_,base_table_path_MH86,_,_ = h.return_local_paths(drive = drive,pre = 'MH86',add_path = True)

In [ ]:
base_table_path_MH86

In [ ]:
df_dge_M_final = pd.read_csv(os.path.join(base_table_path_MH86,'MH86_df_dge_M_final.csv'),index_col=0)
df_dge_M_final = df_dge_M_final[df_dge_M_final['FDR_0.2']]

In [ ]:
species = 'cyno'
conditions  = ['timepoints06hr','timepoints24hr']
celltypes_cyno = ['CD4_T', 'CD14_Mono', 'NKProliferating', 'CD8_T', 'B', 'MAIT', 'dnT']

In [ ]:
all_filtered_dfs_cyno = []
for celltype in celltypes_cyno:
    for condition in conditions:
        df_dge_M_final_ct_cond = df_dge_M_final[(df_dge_M_final['celltype'] == celltype)&(df_dge_M_final['comparison.vs.00hr'] == condition)] 
        df_dge_M_final_ct_cond['abs_logfoldchanges'] = np.abs(df_dge_M_final_ct_cond['logFC'])
        df_dge_M_final_ct_cond = df_dge_M_final_ct_cond.sort_values(by='logFC', ascending=False)
        df_dge_M_final_ct_cond.to_csv(os.path.join(base_table_path,pre + '_df_dge_M_' + str(celltype) + '_' + str(condition) + '_' +str(species) + '.csv'))
        top_200_genes = df_dge_M_final_ct_cond.head(200)['genes'].tolist()
        #df_dge_M_final_ct = df_dge_M_final[df_dge_M_final['celltype'] == celltype]
        
        enrichment = gseapy.enrichr(gene_list=top_200_genes,
             gene_sets=['Reactome_Pathways_2024','WikiPathway_2023_Human','GO_Biological_Process_2023','KEGG_2021_Human','KEGG_2016'],
             organism='Human', 
             cutoff=1
             )
        
        #at this point dont drop duplicate pathways
        df = enrichment.results
        df['species'] = species
        df['celltype'] = celltype
        df['comparison.vs.00hr'] = condition
        df.to_csv(os.path.join(base_table_path,pre + '_df_pathways_' + str(celltype) + '_' + str(condition) + '_' +str(species) + '.csv'))
        filtered_df = df[df['Adjusted P-value'] < 0.05]
        filtered_df['Foreground Count'] = filtered_df['Overlap'].apply(lambda x: int(x.split('/')[0]))
        filtered_df = filtered_df[filtered_df['Foreground Count'] >= 2]
        filtered_df = filtered_df.sort_values(by='Odds Ratio', ascending=False)
        filtered_df['Log Odds Ratio'] = np.log(filtered_df['Odds Ratio'])
        filtered_df = filtered_df.drop_duplicates(subset=['Genes'], keep='first')
        filtered_df['short_term'] = filtered_df['Term'].apply(remove_go)
        filtered_df.to_csv(os.path.join(base_table_path,pre + '_filtered_df_pathways_' + str(celltype) + '_' + str(condition) + '_' +str(species) + '.csv'))
        all_filtered_dfs_cyno.append(filtered_df)

In [ ]:
species = 'human'
conditions  = ['timepoints06hr','timepoints24hr']
celltypes_human = ['CD4_T', 'CD14_Mono', 'NKProliferating', 'CD8_T', 'B', 'MAIT']

In [ ]:
_,base_table_path_H82,_,_ = h.return_local_paths(drive = drive,pre = "H82",add_path = False)
df_dge_H = pd.read_csv(os.path.join(base_table_path_H82,'H82_dge_tab_edgeR.csv'),index_col=0)

df_dge_H['FDR_0.25'] = df_dge_H['FDR'] <= 0.25
df_dge_H['FDR_0.5'] = df_dge_H['FDR'] <= 0.5
df_dge_H['FDR_0.2'] = df_dge_H['FDR'] <= 0.2

In [ ]:
all_filtered_dfs_human = []
for celltype in celltypes_human:
    for condition in conditions:
        df_dge_H_ct_cond = df_dge_H[(df_dge_H['celltype'] == celltype)&(df_dge_H['comparison.vs.00hr'] == condition)] 
        df_dge_H_ct_cond['abs_logfoldchanges'] = np.abs(df_dge_H_ct_cond['logFC'])
        df_dge_H_ct_cond = df_dge_H_ct_cond.sort_values(by='logFC', ascending=False)
        df_dge_H_ct_cond.to_csv(os.path.join(base_table_path,pre + '_df_dge_M_' + str(celltype) + '_' + str(condition) + '_' +str(species) + '.csv'))
        top_200_genes = df_dge_H_ct_cond.head(200)['genes'].tolist()
        #df_dge_H_ct = df_dge_H[df_dge_H['celltype'] == celltype]
        
        enrichment = gseapy.enrichr(gene_list=top_200_genes,
             gene_sets=['Reactome_Pathways_2024','WikiPathway_2023_Human','GO_Biological_Process_2023','KEGG_2021_Human','KEGG_2016'],
             organism='Human', 
             cutoff=0.05
             )
        
        #at this point dont drop duplicate pathways
        df = enrichment.results
        df['species'] = species
        df['celltype'] = celltype
        df['comparison.vs.00hr'] = condition
        df.to_csv(os.path.join(base_table_path,pre + '_df_pathways_' + str(celltype) + '_' + str(condition) + '_' +str(species) + '.csv'))
        filtered_df = df[df['Adjusted P-value'] < 0.05]
        filtered_df['Foreground Count'] = filtered_df['Overlap'].apply(lambda x: int(x.split('/')[0]))
        filtered_df = filtered_df[filtered_df['Foreground Count'] >= 2]
        filtered_df = filtered_df.sort_values(by='Odds Ratio', ascending=False)
        filtered_df['Log Odds Ratio'] = np.log(filtered_df['Odds Ratio'])
        filtered_df = filtered_df.drop_duplicates(subset=['Genes'], keep='first')
        filtered_df['short_term'] = filtered_df['Term'].apply(remove_go)
        filtered_df.to_csv(os.path.join(base_table_path,pre + '_filtered_df_pathways_' + str(celltype) + '_' + str(condition) + '_' +str(species) + '.csv'))
        all_filtered_dfs_human.append(filtered_df)

summarize results and save

In [ ]:
all_human_df = pd.concat(all_filtered_dfs_human,ignore_index=True)
all_cyno_df = pd.concat(all_filtered_dfs_cyno,ignore_index=True)
all_df = pd.concat([all_human_df,all_cyno_df],ignore_index=True)
all_df.to_csv(os.path.join(base_table_path,pre + '_all_df_pathways_.csv'))

save session

In [ ]:
base_package_version_path = h.return_package_version_local_path(drive=drive)
h.save_package_versions(base_package_version_path,pre,do_print = True)
h.print_main_versions()